# Implementation of Finite Mixture Model

## Import Needed Filepaths and Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from texas_gerrymandering_hb4.config import FINAL_CSV

## Load Dataset Into Pandas DataFrame

In [ ]:
df = pd.read_csv(FINAL_CSV)

## Define Response Variable `y`

In [ ]:
y = df["dem_share"].values.astype(float)

## Set Up Reproducible Random Number Generator

In [ ]:
np.random.seed(123)